In [13]:
%%capture
!pip install jupyter_ui_poll
!pip install pandas

In [14]:
import time
import ipywidgets as widgets
from jupyter_ui_poll import ui_events
import pandas as pd


event_info = {
    'type': '',
    'description': '',
    'time': -1
}


def wait_for_event(timeout=-1, interval=0.001, max_rate=20, allow_interupt=True):   
    with ui_events() as ui_poll: ui_poll(999)
    start_wait = time.time()

    # set event info to be empty
    # as this is dict we can change entries
    # directly without using
    # the global keyword
    event_info['type'] = ""
    event_info['description'] = ""
    event_info['time'] = -1

    n_proc = int(max_rate*interval)+1
    
    with ui_events() as ui_poll:
        keep_looping = True
        while keep_looping==True:
            # process UI events
            ui_poll(n_proc)

            # end loop if we have waited more than the timeout period
            if (timeout != -1) and (time.time() > start_wait + timeout):
                keep_looping = False
                
            # end loop if event has occured
            if allow_interupt==True and event_info['description']!="":
                keep_looping = False
                
            # add pause before looping
            # to check events again
            time.sleep(interval)
    
    # return event description after wait ends
    # will be set to empty string '' if no event occured
    return event_info

def register_btn_event(btn):
    event_info['type'] = "button click"
    event_info['description'] = btn.description
    event_info['time'] = time.time()
    return
import requests
from bs4 import BeautifulSoup
import json
import time

def send_to_google_form(data_dict, form_url):
    ''' Helper function to upload information to a corresponding google form 
        You are not expected to follow the code within this function!
    '''
    form_id = form_url[34:90]
    view_form_url = f'https://docs.google.com/forms/d/e/{form_id}/viewform'
    post_form_url = f'https://docs.google.com/forms/d/e/{form_id}/formResponse'

    page = requests.get(view_form_url)
    content = BeautifulSoup(page.content, "html.parser").find('script', type='text/javascript')
    content = content.text[27:-1]
    result = json.loads(content)[1][1]
    form_dict = {}
    
    loaded_all = True
    for item in result:
        if item[1] not in data_dict:
            print(f"Form item {item[1]} not found. Data not uploaded.")
            loaded_all = False
            return False
        form_dict[f'entry.{item[4][0][0]}'] = data_dict[item[1]]
    
    post_result = requests.post(post_form_url, data=form_dict)
    return post_result.ok


from IPython.display import display, Image, clear_output, HTML
import time
import random
import ipywidgets as widgets



data_consent_info = """DATA CONSENT INFORMATION:

Please read:

we wish to record your response data
to an anonymised public data repository. 
Your data will be used for educational teaching purposes
practising data analysis and visualisation.

Please type   yes   in the box below if you consent to the upload."""

print(data_consent_info)
result = input("> ") 

if result == "yes": 
    print("Thanks for your participation.")
    print("Please contact philip.lewis@ucl.ac.uk")
    print("If you have any questions or concerns")
    print("regarding the stored results.")
    
else: 
    # end code execution by raising an exception
    raise(Exception("User did not consent to continue test."))



id_instructions = """

Enter your anonymised ID

To generate an anonymous 4-letter unique user identifier please enter:
- two letters based on the initials (first and last name) of a childhood friend
- two letters based on the initials (first and last name) of a favourite actor / actress

e.g. if your friend was called Charlie Brown and film star was Tom Cruise
     then your unique identifer would be CBTC
"""

print(id_instructions)
user_id = input("> ")

print("User entered id:", user_id)


results_dict = {
    'filename': [],
    'nL': [],
    'nR': [],
    'ratio': [],
    'correct': []
}

picture_paths = ["r20_18_img1.png", "r20_18_img2.png", "l20_18_img1.png", "l20_18_img2.png",
                 "r21_18_img1.png", "r21_18_img2.png", "l21_18_img1.png", "l21_18_img2.png",
                 "r20_15_img1.png", "r20_15_img2.png", "l20_15_img1.png", "l20_15_img2.png",
                 "r16_12_img1.png", "r16_12_img2.png", "l16_12_img1.png", "l16_12_img2.png", 
                 "r18_16_img1.png", "r18_16_img2.png", "l18_16_img1.png", "l18_16_img2.png", 
                 "r14_12_img1.png", "r14_12_img2.png", "l14_12_img1.png", "l14_12_img2.png", 
                 "r12_9_img1.png", "r12_9_img2.png", "l12_9_img1.png", "l12_9_img2.png", 
                 "r10_9_img1.png", "r10_9_img2.png", "l10_9_img1.png", "l10_9_img2.png", 
                ]


def display_dots_test():
    
    results_dict = {}
    results_dict['filename'] = []
    results_dict['correct_side'] = []
    results_dict['user_answer'] = []
    
    #print("What is your name?")
    #name = input()
    print("What is your age?")
    age = input()

    print("What is your sex? (Enter 'm' for Male or 'f' for Female)")
    sex = input().upper()

    clear_output(wait=True)
    
    start_time = time.time()
    score = 0
    
    left_button = widgets.Button(description="Left")
    right_button = widgets.Button(description="Right")
    left_button.on_click(register_btn_event)
    right_button.on_click(register_btn_event)

    # Display buttons
    button_container = widgets.HBox([left_button, right_button])


    print("Set 1 is starting now.")
    time.sleep(4)
    
    set1_picture_paths = picture_paths[:]
    random.shuffle(set1_picture_paths)

    start_time_set1 = time.time()
    score_set1 = 0
    total_questions_set1 = len(set1_picture_paths)

    
    for picture_path in set1_picture_paths:
        if picture_path[0] == 'r':
            answer = 'Right'
        else:
            answer = 'Left'
        picture = Image(filename=picture_path, width=600)

        # Display pictures normally for set 1
        display(picture)
        time.sleep(0.75)
        clear_output(wait=False)

        # Display buttons and wait for user input with a timeout
        display(button_container)
        event_info = wait_for_event(timeout=3)
        print(event_info['description'])
        correct_side = answer

        # Evaluate user response
        if event_info['description'] == correct_side:
            score_set1 += 1
            print("")
        elif event_info['description'] == "":
            print("Too slow")
        else:
            print("")

        # Wait for 1.5 seconds after displaying buttons
        time.sleep(1.5)

        # Store results
        results_dict['filename'].append(picture_path)
        results_dict['correct_side'].append(correct_side)
        results_dict['user_answer'].append(event_info['description'])

    end_time_set1 = time.time()  
    duration_set1 = end_time_set1 - start_time_set1
    accuracy_set1 = (score_set1 / total_questions_set1) * 100
    print(f"\nSet 1 completed in {duration_set1:.2f} seconds.")
    print(f"Score for Set 1: {score_set1}/{total_questions_set1}")
    print(f"Accuracy for Set 1: {accuracy_set1:.2f}%")


    print("Set 1 is over. Set 2 will start now.")
    time.sleep(4)

    set2_picture_paths = picture_paths[:]
    random.shuffle(set2_picture_paths)
    
    start_time_set2 = time.time()
    score_set2 = 0
    total_questions_set2 = len(set2_picture_paths)
    
    # Set 2
    # Shuffle pictures for Set 2


    for picture_path in set2_picture_paths:
        if picture_path[0] == 'r':
            answer = 'Right'
        else:
            answer = 'Left'
        picture = Image(filename=picture_path, width=600)

        # Display pictures for 1.5 seconds for set 2
        display(picture)
        time.sleep(1.5)
        clear_output(wait=False)

        # Display buttons and wait for user input with a timeout
        display(button_container)
        event_info = wait_for_event(timeout=3)
        print(event_info['description'])
        correct_side = answer

        # Evaluate user response
        if event_info['description'] == correct_side:
            score_set2 += 1
            print("")
        elif event_info['description'] == "":
            print("Too slow")
        else:
            print("")

       
        time.sleep(1.5)


        results_dict['filename'].append(picture_path)
        results_dict['correct_side'].append(correct_side)
        results_dict['user_answer'].append(event_info['description'])

    end_time_set2 = time.time()  
    duration_set2 = end_time_set2 - start_time_set2
    accuracy_set2 = (score_set2 / total_questions_set2) * 100
    print(f"\nSet 2 completed in {duration_set2:.2f} seconds.")
    print(f"Score for Set 2: {score_set2}/{total_questions_set2}")
    print(f"Accuracy for Set 2: {accuracy_set2:.2f}%")
    time.sleep(4)
    clear_output(wait=True)

    duration = duration_set1 + duration_set2
    total_score = score_set1 + score_set2
    total_questions = total_questions_set1 + total_questions_set2
    total_accuracy = (total_score / total_questions) * 100
    print(f"Score for Set 1: {score_set1}/{total_questions_set1}")
    print(f"Total time taken for Set 1: {duration_set1:.2f} seconds.")
    print(f"Score for Set 2: {score_set2}/{total_questions_set2}")
    print(f"Total time taken for Set 2: {duration_set2:.2f} seconds.")
    print(f"Total time taken for both sets: {duration:.2f} seconds.")
    print(f"Total score for both sets: {total_score}/{total_questions}")
    
    average_accuracy = (accuracy_set1 + accuracy_set2) / 2
    print(f"Average accuracy for both sets: {average_accuracy:.2f}%")
    
    # Convert results to DataFrame
    results_df = pd.DataFrame(results_dict)
    results_df.index += 1
    display(HTML(results_df.to_html(index=True)))
    results_json = results_df.to_json()
    
    data_dict = {
        'user_id': user_id,
        'age': age,
        'sex': sex,
        'score': total_score,
        'duration': duration,
        'results_json':results_json,
    }

    form_url = "https://docs.google.com/forms/d/e/1FAIpQLSef7pJwfCkuYIOF61a2lRz3mS-6yiDZ_1P6cko9leoQxDUVUQ/viewform?usp=sf_link"  
    send_to_google_form(data_dict, form_url)

    
display_dots_test()

Score for Set 1: 29/32
Total time taken for Set 1: 107.71 seconds.
Score for Set 2: 27/32
Total time taken for Set 2: 126.46 seconds.
Total time taken for both sets: 234.17 seconds.
Total score for both sets: 56/64
Average accuracy for both sets: 87.50%


,filename,correct_side,user_answer
1,l12_9_img1.png,Left,Left
2,r20_15_img1.png,Right,Right
3,r10_9_img1.png,Right,Right
4,r14_12_img1.png,Right,Right
5,l20_15_img1.png,Left,Left
6,r12_9_img2.png,Right,Right
7,l20_18_img1.png,Left,Left
8,l14_12_img1.png,Left,Left
9,r16_12_img2.png,Right,Right
10,l18_16_img2.png,Left,Left
